In [4]:
# PostgreSQLからさまざまなデータを取得する

import os
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
from sqlalchemy import create_engine
from dotenv import load_dotenv
import json
from datetime import datetime

# 環境変数読み込み
load_dotenv()



True

In [5]:
# =============================================================================
# 1. 接続設定
# =============================================================================

print("🔌 データベース接続を設定中...")

# PostgreSQL接続設定
pg_config = {
    'host': os.getenv('POSTGRES_HOST', 'localhost'),
    'database': os.getenv('POSTGRES_DB', 'stallion_db'),
    'user': os.getenv('POSTGRES_USER', 'stallion_user'),
    'password': os.getenv('POSTGRES_PASSWORD'),
    'port': int(os.getenv('POSTGRES_PORT', '5432'))
}

# SQLAlchemy エンジン作成（pandasで使用）
DATABASE_URL = f"postgresql://{pg_config['user']}:{pg_config['password']}@{pg_config['host']}:{pg_config['port']}/{pg_config['database']}"
engine = create_engine(DATABASE_URL)

print("✅ 接続設定完了")

🔌 データベース接続を設定中...
✅ 接続設定完了


In [8]:
# =============================================================================
# pandas.read_sql
# =============================================================================

def query(sql):
    """SQLクエリを実行してDataFrameで結果を返す"""
    return pd.read_sql(sql, engine)

def show_tables():
    """全テーブル一覧を表示"""
    sql = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
    return query(sql)

def desc(table_name):
    """テーブル構造を表示"""
    sql = f"""
    SELECT column_name, data_type, is_nullable, column_default 
    FROM information_schema.columns 
    WHERE table_name = '{table_name}' 
    ORDER BY ordinal_position;
    """
    return query(sql)

def count_all(table_name):
    """テーブルの行数を取得"""
    sql = f"SELECT COUNT(*) as count FROM {table_name};"
    return query(sql).iloc[0, 0]

---
以下、単発のSQLを実行

In [98]:
# queryの中にSQLを書いて実行する

query ("""
      
    SELECT
        jockey_name,
        COUNT(*) AS win_count
    FROM race_results
    WHERE finish_position = 1
    GROUP BY jockey_name
    ORDER BY win_count DESC
    LIMIT 20;
      
    """)

,jockey_name,win_count
0,ルメール,46
1,川田将雅,27
2,Ｍ．デム,24
3,福永祐一,19
4,武豊,18
5,岩田康誠,16
6,池添謙一,16
7,戸崎圭太,12
8,蛯名正義,12
9,横山典弘,9


---
以下SQL練習 with claude

In [99]:
query("""
    WITH jockey_horse_wins AS (
        SELECT 
            jockey_name,
            horse_name,
            COUNT(*) as wins_together
        FROM race_results 
        WHERE finish_position = 1
        GROUP BY jockey_name, horse_name
    ),
    ranked_combinations AS (
        SELECT 
            jockey_name,
            horse_name,
            wins_together,
            ROW_NUMBER() OVER (PARTITION BY jockey_name ORDER BY wins_together DESC) as rank
        FROM jockey_horse_wins
    )
    SELECT 
        jockey_name,
        horse_name,
        wins_together
    FROM ranked_combinations
    WHERE rank = 1
    ORDER BY wins_together DESC
    LIMIT 20;
""")

,jockey_name,horse_name,wins_together
0,ルメール,アーモンドアイ,8
1,武豊,キタサンブラック,6
2,池添謙一,オルフェーヴル,6
3,福永祐一,コントレイル,5
4,蛯名正義,アパパネ,4
5,川田将雅,リバティアイランド,4
6,岩田康誠,ロードカナロア,4
7,内田博幸,ゴールドシップ,4
8,武幸四郎,メイショウマンボ,3
9,松山弘平,デアリングタクト,3


In [103]:
# 各競馬場での騎手勝利ランキング
# 各競馬場で最も勝利数の多い騎手TOP3を取得してください

query("""
    
    WITH track_jockey_wins AS (
        SELECT
            r.track_name,
            rr.jockey_name,
            COUNT(*) AS wins
        FROM races r
        JOIN race_results rr ON r.race_id = rr.race_id
        WHERE rr.finish_position = 1
        GROUP BY r.track_name, rr.jockey_name
    ),
    ranked_jockeys AS (
        SELECT
            track_name,
            jockey_name,
            wins,
            ROW_NUMBER() OVER (PARTITION BY track_name ORDER BY wins DESC) AS rank
        FROM track_jockey_wins
    )
    SELECT
        track_name,
        jockey_name,
        wins
    FROM ranked_jockeys
    WHERE rank <= 3
    ORDER BY track_name, rank;
      
""")

,track_name,jockey_name,wins
0,中京,福永祐一,3
1,中京,藤田伸二,3
2,中京,武豊,2
3,中京1,幸英明,1
4,中山,武豊,8
5,中山,福永祐一,7
6,中山,ルメール,7
7,京都,武豊,21
8,京都,ルメール,12
9,京都,横山典弘,9


In [112]:
# 17.馬の連続勝利記録
# 各馬の最新5戦の成績（着順）を時系列順で取得してください

query("""
    WITH horse_total_prize AS (
        SELECT 
            id as horse_id,
            name_ja,
            COALESCE((profile->>'total_prize_central')::bigint, 0) as total_prize
        FROM horses
        WHERE profile->>'total_prize_central' IS NOT NULL
    ),
    horse_prize_ranking AS (
        SELECT 
            horse_id,
            name_ja,
            total_prize,
            ROW_NUMBER() OVER (ORDER BY total_prize DESC) as prize_rank
        FROM horse_total_prize
        ORDER BY total_prize DESC
        LIMIT 20
    ),
    latest_5_races_per_horse AS (
        SELECT
            rr.horse_id,
            rr.horse_name,
            r.track_name,
            r.race_name,
            r.race_date,
            rr.finish_position,
            rr.popularity,
            rr.jockey_name,
            ROW_NUMBER() OVER(PARTITION BY rr.horse_id ORDER BY r.race_date DESC) AS race_rank
        FROM race_results rr
        JOIN races r ON rr.race_id = r.race_id
        WHERE rr.finish_position IS NOT NULL
    )
    SELECT
        hpr.prize_rank,
        hpr.name_ja,
        hpr.total_prize,
        lr.track_name,
        lr.race_name,
        lr.race_date,
        lr.finish_position,
        lr.popularity,
        lr.jockey_name,
        lr.race_rank
    FROM horse_prize_ranking hpr
    JOIN latest_5_races_per_horse lr ON hpr.horse_id = lr.horse_id
    WHERE lr.race_rank <= 5
    ORDER BY hpr.prize_rank, lr.race_rank;
""")

,prize_rank,name_ja,total_prize,track_name,race_name,race_date,finish_position,popularity,jockey_name,race_rank
0,1,キタサンブラック,180684,中山,第62回有馬記念(GI),2017-12-24,1,1,武豊,1
1,1,キタサンブラック,180684,東京,第37回ジャパンカップ(GI),2017-11-26,3,1,武豊,2
2,1,キタサンブラック,180684,東京,第156回天皇賞(秋)(GI),2017-10-29,1,1,武豊,3
3,1,キタサンブラック,180684,阪神,第58回宝塚記念(GI),2017-06-25,9,1,武豊,4
4,1,キタサンブラック,180684,京都,第155回天皇賞(春)(GI),2017-04-30,1,1,武豊,5
...,...,...,...,...,...,...,...,...,...,...
95,20,ダイワメジャー,100223,中山,第52回有馬記念(GI),2007-12-23,3,6,Ｍデムー,1
96,20,ダイワメジャー,100223,京都,第24回マイルチャンピオンS(GI),2007-11-18,1,1,安藤勝己,2
97,20,ダイワメジャー,100223,東京,第136回天皇賞(秋)(GI),2007-10-28,9,3,安藤勝己,3
98,20,ダイワメジャー,100223,阪神,第48回宝塚記念(GI),2007-06-24,12,5,安藤勝己,4


In [115]:

# 18. レースの人気と着順の関係
# 各レースで人気順位と着順の差を計算してください（大波乱度分析）

query ("""

    WITH upset_analysis AS (
        SELECT
            rr.race_id,
            r.race_name,
            r.race_date,
            rr.horse_name,
            rr.popularity,
            rr.finish_position, 
            rr.popularity - rr.finish_position AS upset_degree,
            ABS(rr.popularity - rr.finish_position) AS upset_magnitude,
            CASE
                WHEN rr.finish_position < rr.popularity THEN '好走'
                WHEN rr.finish_position = rr.popularity THEN '人気通り'
                ELSE '凡走'
            END AS performance_type
        FROM race_results rr
        JOIN races r ON r.race_id = rr.race_id
        WHERE rr.popularity IS NOT NULL
            AND rr.finish_position IS NOT NULL
    )
    SELECT
        *,
        CASE
            WHEN upset_magnitude >= 6 THEN '大波乱'
            WHEN upset_magnitude >= 3 THEN '波乱'
            ELSE '順当'
        END AS upset_level
    FROM upset_analysis
    WHERE upset_analysis.finish_position = 1
    ORDER BY race_date DESC
    LIMIT 50;
       


    """)

,race_id,race_name,race_date,horse_name,popularity,finish_position,upset_degree,upset_magnitude,performance_type,upset_level
0,202509030411,第66回宝塚記念(GI),2025-06-15,メイショウタバル,7,1,6,6,好走,大波乱
1,202505030211,第75回安田記念(GI),2025-06-08,ジャンタルマンタル,2,1,1,1,好走,順当
2,202505021211,第92回東京優駿(GI),2025-06-01,クロワデュノール,1,1,0,0,人気通り,順当
3,202505021011,第86回優駿牝馬(GI),2025-05-25,カムニャック,4,1,3,3,好走,波乱
4,202505020811,第20回ヴィクトリアマイル(GI),2025-05-18,アスコリピチェーノ,1,1,0,0,人気通り,順当
5,202505020611,第30回NHKマイルカップ(GI),2025-05-11,パンジャタワー,9,1,8,8,好走,大波乱
6,202508020411,第171回天皇賞(春)(GI),2025-05-04,ヘデントール,1,1,0,0,人気通り,順当
7,202506030811,第85回皐月賞(GI),2025-04-20,ミュージアムマイル,3,1,2,2,好走,順当
8,202509020611,第85回桜花賞(GI),2025-04-13,エンブロイダリー,3,1,2,2,好走,順当
9,202509020411,第69回大阪杯(GI),2025-04-06,ベラジオオペラ,2,1,1,1,好走,順当


In [119]:
# 📊 レベル8: 複雑な集計・分析
# 距離適性分析
# 各馬が最も勝率の高い距離帯（1200m以下、1201-1600m、1601-2000m、2001m以上）を判定してください

# 17.馬の連続勝利記録
# 各馬の最新5戦の成績（着順）を時系列順で取得してください

query("""
      
    WITH horse_distance_performance AS (
        SELECT
            rr.horse_id,
            rr.horse_name, 
            CASE
                WHEN r.distance <= 1400 THEN '短距離'
                WHEN r.distance BETWEEN 1401 AND 1800 THEN 'マイル'
                WHEN r.distance BETWEEN 1801 AND 2400 THEN '中距離'
                ELSE '長距離'
            END AS distance_category,
            COUNT (*) AS total_runs,
            COUNT (CASE WHEN rr.finish_position = 1 THEN 1 END) AS wins,
            ROUND (
                (CASE WHEN rr.finish_position = 1 THEN 1 END) * 100.0 / COUNT(*),
                2
            ) AS win_rate
        FROM race_results rr
        JOIN races r ON rr.race_id = r.race_id
        GROUP BY rr.horse_id, distance_category
        HAVING COUNT(*) >= 3
    ),
    best_distance_per_horse AS (
        SELECT
            horse_id,
            horse_name,
            distance_category,
            total_runs,
            wins,
            win_rate,
            ROW_NUMBER() OVER (PARTITION BY horse_id ORDER BY win_rate DESC, toral_runs DESC) AS rank
        FROM horse_distance_performance
    )
    SELECT
        h.name_ja AS horse_name,
        bdph.distance_category AS best_distance,
        bdph.total_runs,
        bdph.wins,
        bdph.win_rate,
        (h.profile->>'total_prize_central')::bigint AS total_prize
    FROM best_distance_per_horse bdph
    JOIN horses h ON bdph.horse_id = h.id
    WHERE bdph.rank = 1
        AND (h.profile->>'total_prize_central') IS NOT NULL
        AND (h.profile->>'total_prize_central')::bigint != '0'
    ORDER BY (h.profile->>'total_prize_central')::BIGINT DESC
    LIMIT 20;
    
""")

ProgrammingError: (psycopg2.errors.UndefinedFunction) operator does not exist: character varying <= integer
LINE 8:                 WHEN r.distance <= 1400 THEN '短距離'
                                        ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: 

    WITH horse_distance_performance AS (
        SELECT
            rr.horse_id,
            rr.horse_name, 
            CASE
                WHEN r.distance <= 1400 THEN '短距離'
                WHEN r.distance BETWEEN 1401 AND 1800 THEN 'マイル'
                WHEN r.distance BETWEEN 1801 AND 2400 THEN '中距離'
                ELSE '長距離'
            END AS distance_category,
            COUNT (*) AS total_runs,
            COUNT (CASE WHEN rr.finish_position = 1 THEN 1 END) AS wins,
            ROUND (
                wins * 100.0 / COUNT(*),
                2
            ) AS win_rate
        FROM race_results rr
        JOIN races r ON rr.race_id = r.race_id
        GROUP 